In [2]:
import pandas as pd
import numpy as np
# import tensorflow_datasets as tfds
import math

## Loading the dataset

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    "hihi Ananas",
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
# Parameters
image_train_time = 350

min_frequency = 1
max_frequency = 50

In [5]:
def encode_image_to_spike_train(image: np.ndarray):
    spike_trains = []

    for x_position in range(image.shape[0]):
        for y_position in range(image.shape[1]):

            pixel_value = image[x_position][y_position]

            spike_train = np.zeros(shape=(image_train_time,))

            # Transfer pixel value to set frequency range(and some other stuff, which interp does...)
            frequency = np.interp(pixel_value, [np.min(image),np.max(image)], [min_frequency,max_frequency])

            spike_time_distance = math.ceil(image_train_time / frequency)
            next_spike_time = spike_time_distance

            if pixel_value > 0:
                while next_spike_time < image_train_time + 1:
                    # Add Spike to Spike Train
                    spike_train[int(next_spike_time)] = 1

                    # Calculate next spike
                    next_spike_time += spike_time_distance

            spike_trains.append(spike_train)

    return spike_trains

In [18]:
def receptive_field(image: np.ndarray):
	image_size_x = image.shape[0]
	image_size_y = image.shape[1]

	weight1 =  0.625
	weight2 =  0.125
	weight3 = -0.125
	weight4 = -.5

	# Receptive Field Kernel
	receptive_field = [
        [	weight4 ,weight3 , weight2 ,weight3 ,weight4],
	 	[	weight3 ,weight2 , weight1 ,weight2 ,weight3],
	 	[ 	weight2 ,weight1 , 	  1    ,weight1 ,weight2],
	 	[	weight3 ,weight2 , weight1 ,weight2 ,weight3],
	 	[	weight4 ,weight3 , weight2 ,weight3 ,weight4]]

	convoluted_image = np.zeros(image.shape)
	
	window = [-2,-1,0,1,2]
	x_offset = 2
	y_offset = 2

	# Apply Convolution with Receptive Field Kernel
	for x_image_index in range(image_size_x):
		for y_image_index in range(image_size_y):
			summation = 0
			for x_kernel_index in window:
				for y_kernel_index in window:
					if (x_image_index+x_kernel_index)>=0 and (x_image_index+x_kernel_index)<=image_size_x-1 and (y_image_index+y_kernel_index)>=0 and (y_image_index+y_kernel_index)<=image_size_y-1:
						summation = summation + (receptive_field[x_offset+x_kernel_index][y_offset+y_kernel_index] * image[x_image_index+x_kernel_index][y_image_index+y_kernel_index]) / 255
			convoluted_image[x_image_index][y_image_index] = summation
	return convoluted_image		

[[0.31309044 0.2647257  0.67309367 ... 0.74081758 0.48496389 0.10939674]
 [0.54331181 0.63007093 0.78315876 ... 0.13068327 0.70832273 0.1606179 ]
 [0.98607492 0.83397699 0.68901231 ... 0.8849151  0.65529952 0.29054046]
 ...
 [0.20880216 0.04686101 0.47738489 ... 0.43477288 0.27921282 0.03370032]
 [0.28520354 0.3600988  0.57851659 ... 0.2267577  0.44941091 0.08615663]
 [0.48630197 0.62781075 0.05347851 ... 0.37481318 0.27687128 0.38674573]]


array([[0.00218675, 0.00400979, 0.00521498, ..., 0.00529539, 0.00508217,
        0.00074369],
       [0.00638069, 0.0092308 , 0.00887136, ..., 0.00384143, 0.00544174,
        0.00292229],
       [0.0070326 , 0.00948442, 0.00629501, ..., 0.00272637, 0.00584993,
        0.00303252],
       ...,
       [0.00040458, 0.00275533, 0.00427473, ..., 0.00145242, 0.00309528,
        0.00150383],
       [0.00225543, 0.00448101, 0.00598318, ..., 0.00244967, 0.00252788,
        0.00140274],
       [0.00320733, 0.00319611, 0.00455583, ..., 0.00374876, 0.00325445,
        0.00172645]])

: 

: 

: 